In [12]:
import numpy as np
from scipy.stats import unitary_group

np.set_printoptions(precision=3, suppress=True) #print less digits of floats for readability

In [26]:
def M(n, sigma, delta, m):
    r"""The Bell M matrix from Eq 1 of the paper
    n here represents the starting mode of MZI, and n = k-j
    """
    mat = np.identity(m, dtype=np.complex128)
    mat[n, n] = np.exp(1j * sigma) * np.sin(delta)
    mat[n, n+1] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n+1] = -np.exp(1j * sigma) * np.sin(delta)
    return mat

def P(j, phi, m):
    mat = np.identity(m, dtype=np.complex128)
    mat[j,j] = np.exp(1j * phi)
    return mat

In [27]:
def is_unitary(U):
    return np.allclose(U @ U.conj().T, np.eye(U.shape[0]))

# Reck

In [51]:
m = 3
# V = np.random.random((5,5)) + 1j*np.random.random((5,5))
U = unitary_group.rvs(m)

V = U.conj()

#{"P":[P1,...Pj],
#"Q":[Q1,...Qj],
#"j=0":[[delta_0,sigma_0],],
#"j=1":[[delta_0,sigma_0],[delta_1,sigma_1]]}
#...
params_list = {}
params_list["P_"] = []
params_list["Q_"] = []
params_list["global_zeta"] = []
for j in range(m-1):
    params_list[j] = []

for j in range(m-1):
    x = m - 1
    y = j
    phi_j = np.angle(V[x, y+1]) - np.angle(V[x, y])
    Pj = P(j, phi_j, m)
    params_list["P_"].append(phi_j)
    V = V @ Pj
    for k in range(j+1):
        delta = np.arctan(-V[x,y+1] / V[x,y]).real
        V_temp = V @ M(j-k, 0, delta, m)
        sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
        params_list[j].append([delta,sigma])
        n = j - k
        V = V @ M(n, sigma, delta, m)
        x -= 1
        y -= 1
    
# these next two lines are just to remove a global phase
zeta = - np.angle(V[0,0])
params_list["global_zeta"].append(zeta)
V = V @ P(0, zeta, m)

for j in range(1,m):
    zeta = np.angle(V[0,0]) - np.angle(V[j,j])
    params_list["Q_"].append(zeta)
    V = V @ P(j, zeta, m)
        
V

array([[ 1.-0.j, -0.+0.j,  0.-0.j],
       [ 0.+0.j,  1.+0.j,  0.-0.j],
       [-0.-0.j,  0.+0.j,  1.+0.j]])

In [52]:
U

array([[ 0.126+0.674j, -0.546+0.399j,  0.189+0.19j ],
       [-0.054-0.13j , -0.151-0.263j,  0.9  -0.281j],
       [ 0.334-0.631j, -0.67 +0.03j , -0.127+0.154j]])

In [53]:
params_list

{'P_': [-4.181059803700268, -3.9874535769699815],
 'Q_': [1.7652584411789625, 2.2603512265750845],
 'global_zeta': [2.8545487599612014],
 0: [[-0.7543170115592251, -1.4585709693923294]],
 1: [[-0.20059508000324766, -1.6653345369377348e-16],
  [-1.2935983230717436, -2.0684553771222953]]}

In [55]:
#P(j,phi,m)
#M(n, sigma, delta, m)
m=3
U = np.identity(m, dtype=np.complex128)
for j in range(m-1):
    phi = params_list["P_"][j]
    U = P(j,phi,m) @ U
    for k in range(len(params_list[j])):
        delta = params_list[j][k][0]
        sigma = params_list[j][k][1]
        U = M(j-k, sigma, delta, m) @ U
U = P(0, params_list["global_zeta"][0], m) @ U
for j in range(1,m):
    zeta = params_list["Q_"][j-1]
    U = P(j, zeta, m) @ U
    
U

array([[ 0.126+0.674j, -0.546+0.399j,  0.189+0.19j ],
       [-0.054-0.13j , -0.151-0.263j,  0.9  -0.281j],
       [ 0.334-0.631j, -0.67 +0.03j , -0.127+0.154j]])

# Clements

In [195]:
m = 7
# V = np.random.random((5,5)) + 1j*np.random.random((5,5))
U = unitary_group.rvs(m)

V = U.conj()

params_list = {}
params_list["P_"] = []
params_list["Q_"] = []
params_list["global_zeta"] = []
for j in range(m-1):
    params_list[j] = []
    
for j in range(m-1):
    #odd case in paper, because we index from 0 not 1
    if j % 2 == 0: 
        x = m - 1
        y = j 
        phi_j = np.angle(V[x, y+1]) - np.angle(V[x, y]) # reversed order from paper
        params_list["P_"].append(phi_j)
        V = V @ P(j, phi_j, m)
        for k in range(j+1):
            delta = np.arctan(-V[x,y+1] / V[x,y]) # flipped from paper
            n = j - k
            V_temp = V @ M(n, 0, delta, m)
            sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
            params_list[j].append([delta,sigma])
            V = V @ M(n, sigma, delta, m)
            x -= 1
            y -= 1
    else:
        x = m - j - 1
        y = 0 
        phi_j = np.angle(V[x-1,y]) - np.angle(V[x,y])
        params_list["P_"].append(phi_j)
        V = P(x, phi_j, m) @ V
        for k in range(j+1):
            delta = np.arctan(V[x-1,y] / V[x,y]) # flipped from paper
            V_temp = M(x-1, 0, delta, m) @ V
            n = m + k - j - 2
            if j != k:
                sigma = (np.angle(V_temp[x+1, y+1]) - np.angle(V_temp[x,y+1]))
            else:
                sigma = 0 
            params_list[j].append([delta,sigma])
            V = M(n, sigma, delta, m) @ V
            x += 1
            y += 1

# these next two lines are just to remove a global phase
zeta = - np.angle(V[0,0])
params_list["global_zeta"].append(zeta)
V = V @ P(0, zeta, m)

for j in range(1,m):
    zeta = np.angle(V[0,0]) - np.angle(V[j,j])
    params_list["Q_"].append(zeta)
    V = V @ P(j, zeta, m)
        
V

array([[ 1.+0.j,  0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j, -0.+0.j],
       [-0.-0.j,  1.+0.j, -0.+0.j, -0.+0.j,  0.-0.j,  0.+0.j, -0.+0.j],
       [ 0.-0.j,  0.-0.j,  1.+0.j, -0.+0.j, -0.+0.j, -0.+0.j, -0.+0.j],
       [ 0.+0.j, -0.-0.j, -0.-0.j,  1.+0.j, -0.+0.j,  0.+0.j,  0.-0.j],
       [-0.+0.j,  0.-0.j,  0.+0.j, -0.-0.j,  1.+0.j, -0.-0.j,  0.+0.j],
       [ 0.+0.j, -0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  1.+0.j,  0.-0.j],
       [-0.+0.j, -0.+0.j, -0.+0.j, -0.+0.j, -0.-0.j, -0.-0.j,  1.+0.j]])

In [196]:
U

array([[-0.033+0.372j, -0.28 +0.309j, -0.204+0.001j, -0.247-0.161j,
         0.261-0.453j, -0.267-0.165j,  0.372+0.216j],
       [ 0.335+0.162j, -0.161+0.161j,  0.18 -0.163j,  0.052-0.707j,
        -0.039-0.013j,  0.063+0.403j, -0.272-0.084j],
       [ 0.651-0.127j, -0.205-0.166j,  0.125-0.091j,  0.253+0.348j,
         0.339-0.21j ,  0.211+0.11j ,  0.137+0.217j],
       [ 0.276+0.348j,  0.135-0.156j,  0.062-0.337j,  0.078-0.164j,
        -0.295+0.231j,  0.197-0.537j,  0.367-0.089j],
       [-0.139-0.106j,  0.012-0.118j,  0.377-0.695j, -0.14 +0.165j,
        -0.159-0.134j, -0.443+0.145j, -0.005+0.148j],
       [-0.086-0.074j,  0.289+0.549j,  0.21 -0.172j,  0.12 +0.033j,
         0.478+0.378j,  0.028+0.116j,  0.323-0.152j],
       [-0.042+0.2j  , -0.365+0.361j,  0.057-0.236j, -0.04 +0.359j,
         0.008-0.103j,  0.235-0.252j, -0.452-0.414j]])

In [197]:
#P(j,phi,m)
#M(n, sigma, delta, m)
U_Q = np.identity(m, dtype=np.complex128)
U_odd = np.identity(m, dtype=np.complex128)
U_even = np.identity(m, dtype=np.complex128)

for j in range(m-1):
    if j % 2 == 0: 
        
        phi = params_list["P_"][j]
        U_odd =  U_odd@P(j,phi,m)
        for k in range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
            U_odd = U_odd@M(j-k, sigma, delta, m) 
    else:
        phi = params_list["P_"][j]
        U_even =    P(m - j - 1,phi,m)@U_even
        for k in range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
            U_even = M(m + k - j - 2, sigma, delta, m)@U_even
        

for j in range(1,m):
    zeta = params_list["Q_"][j-1]
    U_Q = P(j, zeta, m) @ U_Q
    
phase = P(0, params_list["global_zeta"][0], m)

U_right = U_odd@phase@U_Q
U_left = U_even

In [198]:
(U_right@U_left).T

array([[-0.033+0.372j, -0.28 +0.309j, -0.204+0.001j, -0.247-0.161j,
         0.261-0.453j, -0.267-0.165j,  0.372+0.216j],
       [ 0.335+0.162j, -0.161+0.161j,  0.18 -0.163j,  0.052-0.707j,
        -0.039-0.013j,  0.063+0.403j, -0.272-0.084j],
       [ 0.651-0.127j, -0.205-0.166j,  0.125-0.091j,  0.253+0.348j,
         0.339-0.21j ,  0.211+0.11j ,  0.137+0.217j],
       [ 0.276+0.348j,  0.135-0.156j,  0.062-0.337j,  0.078-0.164j,
        -0.295+0.231j,  0.197-0.537j,  0.367-0.089j],
       [-0.139-0.106j,  0.012-0.118j,  0.377-0.695j, -0.14 +0.165j,
        -0.159-0.134j, -0.443+0.145j, -0.005+0.148j],
       [-0.086-0.074j,  0.289+0.549j,  0.21 -0.172j,  0.12 +0.033j,
         0.478+0.378j,  0.028+0.116j,  0.323-0.152j],
       [-0.042+0.2j  , -0.365+0.361j,  0.057-0.236j, -0.04 +0.359j,
         0.008-0.103j,  0.235-0.252j, -0.452-0.414j]])